# Stock Price Modeling using AR & MA Models
### Financial Time Series Analysis Project

This notebook covers:
- Time series visualization
- Stationarity testing (ADF Test)
- ACF & PACF analysis
- Autoregressive (AR) Model
- Moving Average (MA) Model
- ARMA Model
- Model evaluation using AIC, BIC
- Residual diagnostics

In [ ]:
# Install required library (for Colab)
!pip install yfinance statsmodels --quiet

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings('ignore')

## Step 1: Download Stock Data

In [ ]:
ticker = 'AAPL'  # Change to INFY.NS, RELIANCE.NS if needed
data = yf.download(ticker, start='2020-01-01', end='2025-01-01')

df = data[['Close']]
df.head()

## Step 2: Plot Time Series

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df['Close'])
plt.title(f'{ticker} Closing Price')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

## Step 3: Stationarity Test (ADF)

In [ ]:
def adf_test(series):
    result = adfuller(series)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    if result[1] <= 0.05:
        print('Series is Stationary')
    else:
        print('Series is Non-Stationary')

adf_test(df['Close'])

## Step 4: Differencing

In [ ]:
df['Close_diff'] = df['Close'].diff()
df = df.dropna()

plt.figure(figsize=(10,5))
plt.plot(df['Close_diff'])
plt.title('Differenced Series')
plt.show()

adf_test(df['Close_diff'])

## Step 5: ACF & PACF

In [ ]:
plot_acf(df['Close_diff'], lags=30)
plt.show()

plot_pacf(df['Close_diff'], lags=30)
plt.show()

## Step 6: AR Model

In [ ]:
ar_model = ARIMA(df['Close'], order=(2,1,0))
ar_result = ar_model.fit()
print(ar_result.summary())

## Step 7: MA Model

In [ ]:
ma_model = ARIMA(df['Close'], order=(0,1,2))
ma_result = ma_model.fit()
print(ma_result.summary())

## Step 8: ARMA Model

In [ ]:
arma_model = ARIMA(df['Close'], order=(2,1,2))
arma_result = arma_model.fit()
print(arma_result.summary())

## Step 9: Model Comparison

In [ ]:
comparison = pd.DataFrame({
    'Model': ['AR(2)', 'MA(2)', 'ARMA(2,2)'],
    'AIC': [ar_result.aic, ma_result.aic, arma_result.aic],
    'BIC': [ar_result.bic, ma_result.bic, arma_result.bic]
})

comparison

## Step 10: Residual Diagnostics

In [ ]:
residuals = arma_result.resid

plt.figure(figsize=(10,4))
plt.plot(residuals)
plt.title('Residuals')
plt.show()

sns.histplot(residuals, kde=True)
plt.title('Residual Distribution')
plt.show()

plot_acf(residuals, lags=30)
plt.show()